In [ ]:
import os
os.getcwd() 

In [ ]:
# Make sure the source code auto reloads into the kernel\n
%load_ext autoreload
%autoreload 2

In [ ]:
from src.utils.logger import logger

In [ ]:
# TODO: Apply DBSCAN for clustering of the provided data (tune hyperparameters to find the optimal number of clusters)

In [ ]:
# TODO: For each cluster compute the central point and the radius

In [ ]:
# TODO: Store the trained models (TfidfVectorizer, PCA and etc) along with the Classifier information: Class Id, Central point, Radius